In [1]:
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')
from mpl_toolkits import mplot3d as mpl3d
from scipy import signal, stats, linalg, fft
import tqdm

# import our 'specsens' package with the help of 'sys'
import sys
sys.path.insert(0, '..')
import specsens as ss

In [2]:
# mpl.use("pgf")  # to display graphs remove this line
mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
line = 0.75

In [3]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.style.use('seaborn-deep')
import time

from specsens import util


def prints(gens,
           pfas,
           pds,
           theo_pfa,
           theo_pd,
           eng_both,
           eng_noise,
           n=None,
           signal_power=None,
           noise_power=None,
           threshold=None,
           num_bands=1,
           num_est_samples=0,
           no_info=False,
           bins=100,
           hist_only=False):

    plt.figure(figsize=(8, 5.5))

    gs = gridspec.GridSpec(2, 2, height_ratios=[0.8, 1.0])

    ax0 = plt.subplot(gs[0])
    ax0.plot(0,
             pfas[0],
             'o',
             c='C0',
             markersize=1,
             label=r'Generation',
             aa=True)
    for i in range(1, gens):
        inter = np.mean(pfas[0:i])
        ax0.plot(i, inter, 'o', c='C0', markersize=1, aa=True)
    ax0.axhline(np.mean(pfas),
                c='C1',
                ls='--',
                linewidth=1,
                label=r'Simulation',
                aa=True)
    ax0.axhline(theo_pfa, c='C2', linewidth=1, label=r'Theory', aa=True)
    ax0.legend(loc=0)
    ax0.set_xlabel(r'Generations')
    ax0.set_ylabel(r'$\%$')
    ax0.set_title(r'(a) $P_{\textit{FA}}$ (Probability of false alarm)')
    ax0.grid(linewidth=0.5)

    ax1 = plt.subplot(gs[1])
    ax1.plot(0,
             pds[0],
             'o',
             c='C0',
             markersize=1,
             label=r'Generation',
             aa=True)
    for i in range(1, gens):
        inter = np.mean(pds[0:i])
        ax1.plot(i, inter, 'o', c='C0', markersize=1, aa=True)
    ax1.axhline(np.mean(pds),
                c='C1',
                ls='--',
                linewidth=1,
                label=r'Simulation',
                aa=True)
    ax1.axhline(theo_pd, c='C2', linewidth=1, label=r'Theory', aa=True)
    ax1.legend(loc=0)
    ax1.set_xlabel(r'Generations')
    ax1.set_ylabel(r'$\%$')
    ax1.set_title(r'(b) $P_D$ (Probability of detection)')
    ax1.grid(linewidth=0.5)

    gs = gridspec.GridSpec(2, 1, height_ratios=[0.8, 1.0])
    ax2 = plt.subplot(gs[1])

    # histogram if we had no information about what is what
    if no_info:
        ax2.hist(np.concatenate((eng_both, eng_noise)),
                 bins,
                 density=True,
                 color='black',
                 alpha=0.5,
                 aa=True,
                 label=r'No information')

    # histograms from simulations
    ax2.hist(eng_both,
             bins,
             density=True,
             color='C0',
             alpha=0.6,
             aa=True,
             label=r'Sim. (Sig present)')
    ax2.hist(eng_noise,
             bins,
             density=True,
             color='C1',
             alpha=0.6,
             aa=True,
             label=r'Sim. (Noise only)')
    ax2.axvline(threshold,
                c='C2',
                ls='--',
                linewidth=1,
                aa=True,
                label=r'Threshold')

    if not hist_only:
        x = np.linspace(np.amin(np.concatenate((eng_both, eng_noise))),
                        np.amax(np.concatenate((eng_both, eng_noise))), 1000)

        # CLT pdfs
        if num_est_samples > 0:  # use estimation stats when using estimation
            snr = util.dB_to_factor_power(
                signal_power) / util.dB_to_factor_power(noise_power)
            snr *= num_bands
            noise_dist = stats.norm.pdf(x,
                                        loc=1,
                                        scale=np.sqrt((n + num_est_samples) /
                                                      (n * num_est_samples)))
            both_dist = stats.norm.pdf(
                x,
                loc=1. + snr,
                scale=np.sqrt(
                    (n + num_est_samples) / (n * num_est_samples)) * (1 + snr))

        else:  # use the regular stats otherwise
            noise_pow = util.dB_to_factor_power(noise_power)
            both_pow = noise_pow + util.dB_to_factor_power(
                signal_power) * num_bands

            #             noise_dist = stats.norm.pdf(x,
            #                                         loc=n * noise_pow,
            #                                         scale=np.sqrt(n * noise_pow**2))
            #             both_dist = stats.norm.pdf(x,
            #                                        loc=n * both_pow,
            #                                        scale=np.sqrt(n * both_pow**2))

            # Chi2 pdfs (not usable for noise estimation)
            noise_dist = stats.chi2.pdf(x,
                                        df=2. * n,
                                        loc=0.,
                                        scale=noise_pow / 2.)
            both_dist = stats.chi2.pdf(x,
                                       df=2. * n,
                                       loc=0.,
                                       scale=both_pow / 2.)
        ax2.plot(x,
                 both_dist,
                 c='C3',
                 linewidth=1,
                 aa=True,
                 label=r'Theory (Sig present)')
        ax2.plot(x,
                 noise_dist,
                 c='C4',
                 linewidth=1,
                 aa=True,
                 label=r'Theory (Noise only)')

    ax2.legend(loc=0)
    ax2.grid(linewidth=0.5)
#     ax2.set_xlabel(r'')
    ax2.set_title(r'(c) Test statistic distribution')
    ax2.set_ylabel(r'$\%$')

    plt.tight_layout()
    plt.savefig('out/chap5/eigenvalue_sim.pdf')

    print('---- Distribution stats ----')
    print('Sig present mean: %.4f' % (np.mean(eng_both)))
    print('Sig absent  mean: %.4f' % (np.mean(eng_noise)))
    print('Sig present var:  %.4f' % (np.var(eng_both)))
    print('Sig absent  var:  %.4f' % (np.var(eng_noise)))
    print('Sig present std:  %.4f' % (np.std(eng_both)))
    print('Sig absent  std:  %.4f' % (np.std(eng_noise)))


import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-deep')
import multiprocessing as mp
import tqdm
from functools import partial

from specsens import util
from specsens import util_sim
from specsens import WirelessMicrophone
from specsens import WhiteGaussianNoise
from specsens import eigen_detector
from specsens import eigen_stats


def generation(f_sample, length_sec, itrs, noise_power, signal_power,
               noise_uncert, threshold, f_center, cov_size, detector_type,
               seeds):

    # create new signal objects
    wm = WirelessMicrophone(f_sample=f_sample, t_sec=length_sec, seed=seeds[0])
    wgn = WhiteGaussianNoise(f_sample=f_sample,
                             t_sec=length_sec,
                             seed=seeds[1])

    # local rng
    rng = np.random.default_rng(seeds[2])

    # calculate noise power with uncertainty
    gen_noise_power = rng.normal(loc=noise_power, scale=noise_uncert)

    # store results in 'result' array energies in 'energy' array
    result = np.array([])
    energy = np.array([])

    # list for noise estimation
    noise_est_list = np.array([])

    # 'inner' interations loop
    for _ in range(itrs):

        # generate signal
        sig = wm.soft(f_center=f_center, power=signal_power, dB=True)

        # generate noise
        noise = wgn.signal(power=gen_noise_power, dB=True)

        # randomly decide whether signal should be present
        sig_present = rng.choice([True, False])
        if sig_present:
            both = sig + noise
        else:
            both = noise

        if detector_type == 'eme':
            # energy with minimum eigenvalue
            eng = eigen_detector.eme(x=both, l=cov_size)
        else:
            # maximum-minimum eigenvalue detector
            eng = eigen_detector.mme(x=both, l=cov_size)

        # threshold
        sig_detected = eng > threshold

        # log detection outcome
        if sig_present and sig_detected:
            result = np.append(result, 1)
        elif sig_present and not sig_detected:
            result = np.append(result, 2)
        elif not sig_present and sig_detected:
            result = np.append(result, 3)
        else:
            result = np.append(result, 4)

        # log energy
        energy = np.append(energy, eng)

    # calculate statistics and store in arrays
    pfa_tmp = np.sum(result == 3) / (np.sum(result == 3) + np.sum(result == 4))
    pd_tmp = np.sum(result == 1) / (np.sum(result == 1) + np.sum(result == 2))

    return pfa_tmp, pd_tmp, energy, result, gen_noise_power


def eigenvalue_sim(
    gens=50,  # generations, number of environments
    itrs=300,  # iterations, number of tests in each environment
    f_sample=1e6,  # in Hz
    signal_power=0.,  # in dB
    f_center=-1e5,  # signal center frequency
    noise_power=0.,  # in dB
    length_sec=None,  # length of each section in seconds
    num_samples=None,  # number of samples
    theo_pfa=0.1,  # probability of false alarm
    threshold=None,  # threshold used for detection
    noise_uncert=0.0,  # standard deviation of the noise normal distribution
    seed=None,  # random seed used for rng
    num_procs=None,  # number of processes to run in parallel
    cov_size=10,  # covariance matrix size
    sampling_factor=1.,  # sampling factor
    detector_type='mme'):  # select whether to use mme or eme detector

    # set number of processes used
    if num_procs is None:
        num_procs = mp.cpu_count()
    assert num_procs > 0, 'num_procs must be greater than 0'
    assert num_procs <= gens, 'num_procs must be less or equal to gens'

    # check and calculate length (in seconds and number of samples)
    if num_samples is not None:
        assert num_samples > 0., 'num_samples must be greater than 0'
        length_sec = num_samples / f_sample
    elif length_sec is not None:
        assert length_sec > 0., 'length_sec must be greater than 0'
        length_sec = length_sec
        num_samples = int(f_sample * length_sec)
    else:
        assert False, 'either num_samples or length_sec needed'

    # calculate threshold
    if threshold is None:
        if detector_type == 'eme':
            threshold = eigen_stats.eme_thr(Ns=num_samples,
                                            L=cov_size,
                                            Pfa=theo_pfa,
                                            M=sampling_factor)
        else:
            threshold = eigen_stats.mme_thr(Ns=num_samples,
                                            L=cov_size,
                                            Pfa=theo_pfa,
                                            M=sampling_factor)

    print('---- Simulation parameters ----')
    print('Generations:    %d' % (gens))
    print('Iterations:     %d' % (itrs))
    print('Total iters:    %d' % (gens * itrs))
    print('Signal power:   %.2f dB' % (signal_power))
    print('Sig cent. freq: %.1f Hz' % (f_center))
    print('Noise power:    %.2f dB' % (noise_power))
    print('Noise uncert:   %.2f dB' % (noise_uncert))
    print('SNR:            %.2f dB' % (signal_power - noise_power))
    print('Signal length:  %.6f s' % (length_sec))
    print('Signal samples: %d' % (num_samples))
    print('Detector type:  %s' % (detector_type))

    # calculate pd (only needed for prints)
    if detector_type == 'eme':
        theo_pd = float('nan')
    else:
        theo_pd = eigen_stats.mme_pd(Ns=num_samples,
                                     L=cov_size,
                                     thr=1.13247650,
                                     noise_power=noise_power,
                                     sig_eigv_min=1.7329314663932066e-13,
                                     sig_eigv_max=12.142304916966683,
#                                      M=0.31, # @20dB, no uncert
                                     M=0.0004, # @17.5dB, 3dB uncert
                                     dB=True)

    print('---- Simulation stats theory ----')
    print('Prob false alarm: %.4f' % (theo_pfa))
    print('Prob detection:   %.4f' % (theo_pd))
    print('Threshold:        %.8f' % (threshold))

    print('---- Running simulation ----')
    print('Using %d processes on %d cores' % (num_procs, mp.cpu_count()))

    pfas = list()  # probability of false alarm list
    pds = list()  # probability of detection list

    # generate child seeds for wm and wgn
    seed_seq = np.random.SeedSequence(seed)
    seeds = list(
        zip(seed_seq.spawn(gens), seed_seq.spawn(gens), seed_seq.spawn(gens)))

    # prepare parallel execution
    p = mp.Pool(processes=num_procs)
    f = partial(generation, f_sample, length_sec, itrs, noise_power,
                signal_power, noise_uncert, threshold, f_center, cov_size,
                detector_type)

    # run simulation while showing progress bar
    res = list(tqdm.tqdm(p.imap(f, seeds), total=gens))

    # cleanup parallel execution
    p.close()
    p.join()

    # 'unwrap' res tuples
    pfas = [r[0] for r in res]
    pds = [r[1] for r in res]
    energies = np.ravel([r[2] for r in res])
    results = np.ravel([r[3] for r in res])
    gen_noise_powers = np.ravel([r[4] for r in res])

    # compute stats from lists
    pfa = np.sum(pfas) / gens
    pd = np.sum(pds) / gens

    # compute energy distributions
    engs_both = energies[np.where(results <= 2)[0]]
    engs_noise = energies[np.where(results > 2)[0]]

    print('---- Simulation stats ----')
    print('Prob false alarm theory: %.4f' % (theo_pfa))
    print('Prob false alarm sim:    %.4f' % (pfa))
    print('Prob detection theory:   %.4f' % (theo_pd))
    print('Prob detection sim:      %.4f' % (pd))

    prints(gens,
           pfas,
           pds,
           theo_pfa,
           theo_pd,
           engs_both,
           engs_noise,
           num_samples,
           threshold=threshold,
           hist_only=True)

    return pfa, pd


_ = eigenvalue_sim(
    gens=200,  # generations, number of environments
    itrs=100,  # iterations, number of tests in each environment
    f_sample=1e6,  # in Hz
    signal_power=1.0,  # in dB
    noise_power=17.5,  # in dB
    length_sec=0.01,  # length of each sample in seconds
    theo_pfa=0.1,  # probability of false alarm
    noise_uncert=3.0,  # standard deviation of the noise normal distribution
    num_procs=None,  # number of processes to run in parallel
    seed=0,  # set seed for reproducibility
    cov_size=10,  # covariance matrix size
    sampling_factor=0.92,  # mme
    detector_type='mme')  # select whether to use mme or eme detector

---- Simulation parameters ----
Generations:    200
Iterations:     100
Total iters:    20000
Signal power:   1.00 dB
Sig cent. freq: -100000.0 Hz
Noise power:    17.50 dB
Noise uncert:   3.00 dB
SNR:            -16.50 dB
Signal length:  0.010000 s
Signal samples: 10000
Detector type:  mme
---- Simulation stats theory ----
Prob false alarm: 0.1000
Prob detection:   0.9656
Threshold:        1.13247650
---- Running simulation ----
Using 8 processes on 8 cores


 38%|███▊      | 75/200 [03:04<05:07,  2.46s/it]Process ForkPoolWorker-2:
Process ForkPoolWorker-8:
Traceback (most recent call last):
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  Fi

KeyboardInterrupt: 

  File "../specsens/eigenvalue_detect/eigen_detector.py", line 16, in corr
    r += np.dot(x[i:i + l], x[i:i + l].conj().T)
KeyboardInterrupt
Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True,